# Python Parsing with NLTK and Foma

**(C) 2017 by [Damir Cavar](http://cavar.me/damir/)**

**License:** [Creative Commons Attribution-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by-sa/4.0/) ([CA BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/))

This is a tutorial related to the discussion of grammar engineering and parsing in the class *Alternative Syntactic Theories* taught at Indiana University in the Linguistics Department in Spring 2017.

The following code examples require an installed *foma.py* module. Since I am using [Python 3.x](https://www.python.org/downloads/) here, I would recommend to use my version of *foma.py* and install it in the local modules folder of your Python distribution. In my case, since I use [Anaconda](https://www.continuum.io/downloads), the file goes in *anaconda/lib/python3.5/* in my home directory.

## Grammars and Parsers

We will use NLTK in the following:

In [2]:
import nltk

We can declare a feature structure and display it using NLTK:

In [3]:
fstr = nltk.FeatStruct("[POS='N', AGR=[PER=3, NUM='pl', GND='fem']]")
print(fstr)

[       [ GND = 'fem' ] ]
[ AGR = [ NUM = 'pl'  ] ]
[       [ PER = 3     ] ]
[                       ]
[ POS = 'N'             ]


We will also use Feature grammars:

In [34]:
from nltk.grammar import FeatureGrammar, FeatDict

We can define a feature grammar in the following way:

In [38]:
grammarText = """
% start S
# ############################
# Grammar Rules
# ############################
S -> NP[NUM=?n, PERS=?p, CASE='nom'] VP[NUM=?n, PERS=?p]
NP[NUM=?n, PERS=?p, CASE=?c] -> N[NUM=?n, PERS=?p, CASE=?c]
NP[NUM=?n, PERS=?p, CASE=?c] -> D[NUM=?n,CASE=?c] NC[NUM=?n,PERS=?p,CASE=?c]
NP[NUM=?n, PERS=?p, CASE=?c] -> Pron[NUM=?n,PERS=?p,CASE=?c]
VP[NUM=?n, PERS=?p] -> V[NUM=?n, PERS=?p]
VP[NUM=?n, PERS=?p] -> V[NUM=?n, PERS=?p] NP[CASE='acc']
"""

grammar = FeatureGrammar.fromstring(grammarText)

Testing the grammar with the input sentence *John loves Mary* fails, because there is not lexical defintion of these entries in the grammar.

In [39]:
parser = nltk.parse.FeatureChartParser(grammar)
sentence = "John loves Mary"
result = list(parser.parse(sentence.split()))
if result:
    for x in result:
        print(x)
else:
    print("*", sentence)

ValueError: Grammar does not cover some of the input words: "'John', 'loves', 'Mary'".

We can include foma and a morphology using the following code:

In [40]:
import foma

The following line will load the *eng.fst* Foma morphology:

In [41]:
fst = foma.FST.load(b'eng.fst')

We can print out the analysis for each single token by submitting it to the FST:

In [42]:
tokens = "John loves Mary".split()
for token in tokens:
    result = list(fst.apply_up(str.encode(token)))
    for r in result:
        print(r.decode('utf8'))

John+N+Sg+Masc+NEPerson
love+V+3P+Sg
Mary+N+Sg+Fem+NEPerson


If we want to convert the flat string annotation from the morphology to a NLTK feature structure, we need to translate some entires to a corresponding Attribute Value Matrix (AVM). In the following table we define a feature in the morphology output and the corresponging feature structure that it corresponds with:

In [43]:
featureMapping = {
    'Def'   : "DETTYPE = def",
    'Indef' : "DETTYPE = indef",
    'Sg'    : "NUM = sg",
    'Pl'    : "NUM = pl",
    '3P'    : "PERS = 3",
    'Masc'  : "GENDSEM = male",
    'Fem'   : "GENDSEM = female",
    'Dat'   : "CASE = dat",
    'Acc'   : "CASE = acc",
    'Nom'   : "CASE = nom",
    'NEPersonName' : """NTYPE = [NSYN = proper,
                                NSEM = [PROPER = [PROPERTYPE = name,
                                                 NAMETYPE   = first_name]]],
                        HUMAN = 1"""
}

We use a function *feat2LFG* to convert the feature tag in the morphological analysis to a LFG-compatible AVM:

In [44]:
def feat2LFG(f):
    result = featureMapping.get(f, "")
    return(nltk.FeatStruct("".join( ("[", result, "]") )))

The following helper function is recursive. It mapps the AVM to a bracketed string annotation of feature structures, as used in the NLTK feature grammar format:

In [45]:
def flatFStructure(f):
    res = ""
    for key in f.keys():
        val = f[key]
        if res:
            res += ', '
        if (isinstance(val, FeatDict)):
            res += key + '=' + flatFStructure(val)
        else:
            res += key + "=" + str(val)
    return('[' + res + ']')

The following function is a parse function that prints out parse trees for an input, maintaining the extended feature structures at the lexical level. It can now parse sentences that contain words that are not specified as lexical words in the grammar, but rather as paths in the morphological finite state transducer.

In [49]:
def parseFoma(sentence):
    tokens = sentence.split()

    tokenAnalyses = {}
    rules = []
    count = 0
    for token in tokens:
        aVal = []
        result = list(fst.apply_up(str.encode(token)))
        for r in result:
            elements = r.decode('utf8').split('+')

            lemma = elements[0]
            tokFeat = nltk.FeatStruct("[PRED=" + lemma + "]")

            cat = elements[1]
            if len(elements) > 2:
                feats = tuple(elements[2:])
            else:
                feats = ()
            for x in feats:
                fRes2 = feat2LFG(x)
                fRes = tokFeat.unify(fRes2)
                if fRes:
                    tokFeat = fRes
                else:
                    print("Error unifying:", tokFeat, fRes2)
            flatFStr = flatFStructure(tokFeat)
            aVal.append(cat + flatFStr)
            rules.append(cat + flatFStr + " -> " + "'" + token + "'")
        tokenAnalyses[count] = aVal
        count += 1

    grammarText2 = grammarText + "\n" + "\n".join(rules)

    grammar = FeatureGrammar.fromstring(grammarText2)
    parser = nltk.parse.FeatureChartParser(grammar)
    result = list(parser.parse(tokens))
    if result:
        for x in result:
            print(x)
    else:
        print("*", sentence)


We can call this function using the following code:

In [50]:
parseFoma("John loves her")

(S[]
  (NP[CASE=?c, NUM='sg', PERS=?p]
    (N[GENDSEM='male', NUM='sg', PRED='John'] John))
  (VP[NUM='sg', PERS=3]
    (V[NUM='sg', PERS=3, PRED='love'] loves)
    (NP[CASE='acc', NUM='sg', PERS=3]
      (Pron[CASE='acc', GENDSEM='female', NUM='sg', PERS=3, PRED='she']
        her))))
